# Introduction

This notebook demonstrates the use of pre-trained MEGNet models to predict properties.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [1]:
import torch
import dgl
import os
from pymatgen.core import Element, Structure, Lattice

## Import megnet related modules
from matgl.graph.converters import get_element_list, Pmg2Graph
from matgl.graph.compute import compute_pair_vector_and_distance
from matgl.models.megnet import MEGNet, MEGNetCalculator


In [2]:
# Let's set the device first. Either "cuda" or "cpu"
device = torch.device("cpu")

# MP Formation energy 

The pre-trained model is traned on the Materaisl Project mp.2018.6.1.json dataset.

In [3]:
# load the pre-trained MEGNet model. By default it is the formation energy model.
model = MEGNet.load()
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, device=device)
eform = predictor.predict_structure(struct)
print(f"The predicted formation energy for a FCC Cu crystal is {float(eform.numpy()):5f} eV/atom")


The predicted formation energy for a FCC Cu crystal is -0.007481 eV/atom


/Users/kenko/miniconda3/envs/mavrl/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  return th.as_tensor(data, dtype=dtype)
/Users/kenko/miniconda3/envs/mavrl/lib/python3.10/site-packages/dgl/readout.py:443: DGLWarning: For a single graph, use a tensor of shape (1, *) for graph_feat. The support of shape (*) will be deprecated.
  dgl_warning(


# MP Band gap

The pre-trained model is traned on the Materails Project mp.2019.4.1.json dataset.

In [4]:
model = MEGNet.load("MP-2019.4.1-BandGap")
# read structure
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.14), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
# define graph label ("0": PBE, "1": GLLB-SC, "2": HSE, "3": SCAN) for multi-fidelity model
graph_attrs = torch.tensor([0])
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, device=device)
bandgap = predictor.predict_structure(struct, graph_attrs)
print(f"The predicted PBE BandGap for a CsCl crystal is {float(bandgap.numpy()):.3f} eV")

The predicted PBE BandGap for a CsCl crystal is -0.007 eV
